# Function Calling in Semantic Kernel

This notebook demonstrates how functions work in the Microsoft Semantic Kernel. 

It shows how the model decides when to execute functions, how functions are invoked, and how Semantic Kernel orchestrates all the steps for function execution. 

You'll learn to define, register, and invoke both inline and C# plugin functions, and see how the AI model and kernel orchestration works to deliver response.

The plugin functions use local mock data to simulate real-world scenarios. In the next lab, you'll learn how to call external APIs and use real data.

**Objectives:**
- Understand how Semantic Kernel orchestrates function execution.
- Define and use inline semantic functions.
- Pass parameters to functions.
- Register and invoke C# plugin (native) functions.


## Setup

In this section, we will set up the Semantic Kernel environment and configure it to use different LLM providers.

**Step 1:** Install NuGet packages

Install the required NuGet packages for Semantic Kernel and DotNetEnv to enable AI-powered workflows and environment variable management.

In [1]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.55.0"
#r "nuget: DotNetEnv, 3.1.0"

Installed Packages DotNetEnv, 3.1.0 Microsoft.SemanticKernel, 1.55.0

**Step 2:** Load environment variables

Load environment variables from a `.env` file if present. This helps manage secrets and configuration settings for your application.

In [2]:
using DotNetEnv;
using System.IO;

var envFilePath = Path.Combine(Environment.CurrentDirectory, "../..", ".env");
if (File.Exists(envFilePath))
{
    Env.Load(envFilePath);
    Console.WriteLine($"Loaded environment variables from {envFilePath}");
}
else
{
    Console.WriteLine($"No .env file found at {envFilePath}");
}

Loaded environment variables from d:\personal\aiagent-workshop\notebooks\semantic-kernel\../..\.env


**Step 3:** Instantiate the Semantic Kernel

Create and configure a Kernel instance, which will be used to interact with AI models.

In [3]:
using System.ClientModel;
using OpenAI;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using System.Text;

OpenAIClient client = null;
if(Environment.GetEnvironmentVariable("USE_AZURE_OPENAI") == "true")
{
    // Configure Azure OpenAI client
    var azureEndpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
    var apiKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY");
    client = new OpenAIClient(new ApiKeyCredential(apiKey), new OpenAIClientOptions { Endpoint = new Uri(azureEndpoint) });
}
else if(Environment.GetEnvironmentVariable("USE_OPENAI") == "true")
{
    // Configure OpenAI client
    var apiKey = Environment.GetEnvironmentVariable("OPENAI_API_KEY");
    client = new OpenAIClient(new ApiKeyCredential(apiKey));
}
else if(Environment.GetEnvironmentVariable("USE_GITHUB") == "true")
{
    // Configure GitHub model client
    var uri = Environment.GetEnvironmentVariable("GITHUB_MODEL_ENDPOINT");
    var apiKey = Environment.GetEnvironmentVariable("GITHUB_TOKEN");
    client = new OpenAIClient(new ApiKeyCredential(apiKey), new OpenAIClientOptions { Endpoint = new Uri(uri) });
}

var modelId = Environment.GetEnvironmentVariable("MODEL");
// Create a chat completion service
var builder = Kernel.CreateBuilder();
builder.AddOpenAIChatCompletion(modelId, client);

// Get the chat completion service
Kernel kernel = builder.Build();

## Calling Functions (Semantic Functions)

This section demonstrates how to define and call functions within the Semantic Kernel. Functions can be defined as text (Semantic) or in C# (native). 

**Step 4:** Define an inline semantic function

Create an inline semantic function using a prompt template. This function will summarize input content, with a configurable word 
count.


Set up the execution settings for the inline function, such as maximum tokens, temperature, and top-p sampling. Then, create the function from the prompt template

In [15]:
string skPrompt = """
{{$input}}

Summarize the features of the bike described above in a maximum of {{$wordCount}} words, focusing on what makes it suitable for different types of riders.

""";

var summaryFunction = kernel.CreateFunctionFromPrompt(skPrompt);


**Step 6:** Prepare input content

Set up some content to summarize. In this example, the input describes the Contoso Hybrid Bike and its features.

In [16]:
var input = """
The Contoso Hybrid Bike combines the lightweight frame of a road bike with the comfort and versatility of a mountain bike. It features a suspension fork for smooth rides on city streets and light trails, reliable disc brakes for all-weather stopping power, and ergonomic grips for long-distance comfort. This bike is ideal for commuters, fitness riders, and anyone who wants a single bike for multiple uses.
""";

**Step 7:** Call the inline function

Send the bike description and word count to the inline function and receive a summary response from the AI model, suitable for a customer-facing product page or quick recommendation.

In [17]:
// Call the kernel to get a response 
var response = await kernel.InvokeAsync(summaryFunction, new() { ["input"] = input, ["wordCount"] = 15 });  
Console.WriteLine($"Response: {response}");

Response: Lightweight, versatile hybrid bike with suspension fork, disc brakes, and ergonomic design for all-purpose riding.


## Calling Native Functions (C# Plugin Functions)
This section demonstrates how LLMs can automatically invoke C# plugin functions based on the input and the model's understanding of the task, such as checking store inventory or providing weather for bike rides.

**Step 1:** Enable automatic function calling

You can configure the execution settings to allow the AI model to automatically call functions based on the prompt, similar to the function calling workflow. This is useful for tasks like checking bike availability or getting weather updates for planning a ride.

In [18]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;

#pragma warning disable SKEXP0001
var openAiPromptSettings = new OpenAIPromptExecutionSettings
{
    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
};
#pragma warning restore SKEXP0001

**Step 2:** Register a C# function and call it automatically

Define and register C# functions (plugins) with the kernel. For the Contoso Bike Store, you might want a function to check bike availability or provide weather information for planning a ride. The AI model can invoke these functions automatically based on the user's prompt.

In [21]:
// Define a plugin for weather information
using System;
using Microsoft.SemanticKernel;
using System.ComponentModel;

public class WeatherPlugin
{
    [KernelFunction("GetWeather")]
    [Description("This function provides a random weather report for a bike ride")]
    public string GetWeather()
    {
        var weatherOptions = new[] { "Sunny", "Cloudy", "Rainy", "Windy", "Stormy", "Snowy" };
        var rnd = new Random();
        var temp = rnd.Next(10, 35);
        var weather = weatherOptions[rnd.Next(weatherOptions.Length)];
        return $"The weather is {weather} and {temp}°C. Perfect for a bike ride!";
    }
}

// Define a plugin for bike store inventory
public class BikeStorePlugin
{
    [KernelFunction("CheckBikeAvailability")]
    [Description("This function checks if a specific bike model is available in the store")]
    public bool CheckBikeAvailability(string model)
    {
        // Simulate inventory lookup
        var availableModels = new[] { "Contoso Explorer", "Contoso CityLite", "Contoso TrailBlazer", "Contoso Speedster" };
        if (Array.Exists(availableModels, m => m.Equals(model, StringComparison.OrdinalIgnoreCase)))
            return true;
        return false;
    }
}

// Register the plugins
var weatherPlugin = new WeatherPlugin();
kernel.ImportPluginFromObject(weatherPlugin, "weather");

var bikeStorePlugin = new BikeStorePlugin();
kernel.ImportPluginFromObject(bikeStorePlugin, "bikeStore");


Error: System.ArgumentException: An item with the same key has already been added. Key: weather
   at System.Collections.Generic.Dictionary`2.TryInsert(TKey key, TValue value, InsertionBehavior behavior)
   at System.Collections.Generic.Dictionary`2.Add(TKey key, TValue value)
   at Microsoft.SemanticKernel.KernelPluginCollection.Add(KernelPlugin plugin)
   at Microsoft.SemanticKernel.KernelExtensions.ImportPluginFromObject(Kernel kernel, Object target, String pluginName)
   at Submission#21.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [22]:
using Microsoft.SemanticKernel.Connectors.OpenAI;

var chat = kernel.GetRequiredService<IChatCompletionService>();
// Create prompts that will trigger the function calls
string prompt1 = "Is the Contoso Speedster bike available in the store?";
string prompt2 = "What is the weather like for a bike ride today?";

var result1 = await chat.GetChatMessageContentAsync(prompt1, openAiPromptSettings, kernel);
Console.WriteLine($"Response: {result1}");




Response: The Contoso Speedster bike is not available in the store.


Now try another example where the model calls the `GetWeather` function to provide weather information for a bike ride.

In [23]:

string prompt2 = "What is the weather like for a bike ride today?";
var result2 = await chat.GetChatMessageContentAsync(prompt2, openAiPromptSettings, kernel);
Console.WriteLine($"Response: {result2}");

Response: The weather today is stormy with a temperature of 29°C. It might not be safe for a bike ride due to the stormy conditions.
